In [1]:
# Run it only once
from datasets import load_dataset
dataset = load_dataset("multi_woz_v22")

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/adrian/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/6719c8b21478299411a0c6fdb7137c3ebab2e6425129af831687fb7851c69eb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
# Run it only once
# dataset.save_to_disk("dataset.hf")

In [3]:
# from datasets import DatasetDict
# dataset = DatasetDict.load_from_disk("dataset.hf")

In [4]:
dir(dataset['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_examples_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_with_indices_mapping',
 '_split',
 'add_column',
 'add

In [5]:
dataset['train'].features

{'dialogue_id': Value(dtype='string', id=None),
 'services': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'turns': Sequence(feature={'turn_id': Value(dtype='string', id=None), 'speaker': ClassLabel(names=['USER', 'SYSTEM'], id=None), 'utterance': Value(dtype='string', id=None), 'frames': Sequence(feature={'service': Value(dtype='string', id=None), 'state': {'active_intent': Value(dtype='string', id=None), 'requested_slots': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'slots_values': Sequence(feature={'slots_values_name': Value(dtype='string', id=None), 'slots_values_list': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, length=-1, id=None)}, 'slots': Sequence(feature={'slot': Value(dtype='string', id=None), 'value': Value(dtype='string', id=None), 'start': Value(dtype='int32', id=None), 'exclusive_end': Value(dtype='int32', id=None), 'copy_from': Value(dtype='string', id=None), 'copy_from_value': Sequence(featur

In [6]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [7]:
dataset['train'][0].keys()

dict_keys(['dialogue_id', 'services', 'turns'])

In [8]:
print(len(dataset["train"]), len(dataset["validation"]), len(dataset["test"]))

8437 1000 1000


In [9]:
set([service for dial in dataset['train'] for service in dial['services']])

{'attraction', 'bus', 'hospital', 'hotel', 'restaurant', 'taxi', 'train'}

In [10]:
set([service for dial in dataset['train'] for turn_id,utt in enumerate(dial['turns']['utterance']) for service in dial['turns']['frames'][turn_id]['service']])

{'attraction',
 'bus',
 'hospital',
 'hotel',
 'police',
 'restaurant',
 'taxi',
 'train'}

In [11]:
dataset['train'][112]

{'dialogue_id': 'SSNG0297.json',
 'services': ['hotel'],
 'turns': {'turn_id': ['0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13'],
  'speaker': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  'utterance': ['Can you find me a 3 star hotel in the centre?',
   'The Gonville Hotel meets your requirements. Would you like me to book you a room?',
   'Does it have free wifi? I need wifi.',
   'Yes, they do offer free internet. Would you like me to make a booking?',
   'Yes. I want to book it for 1 person. I want to stay for 5 nights starting from Wednesday.',
   'I am sorry Booking was unsuccessful,can I help you find another place to stay?',
   'Yes please. Another 3-star hotel in the centre with free wifi.',
   'I do not have another 3 star hotel in the centre that meets your requirements. Would you like to try another area?',
   'Do you have any hotels in the same price range as the Gonville Hotel?',
   'The only other hotel that

In [12]:
from agent import Agent
agent = Agent(intent_use_history = True)

def Dialogue_Act_Prediction(utterance, other_features_from_dialogue_history):
    utterance_history = other_features_from_dialogue_history['utterance_history']
    acts_history = other_features_from_dialogue_history['acts_history']
    return agent.predict_intent(utterance, utterance_history, acts_history)
    # return ['Hotel-Request']

def Extract_and_Categorize_Spans(user_utterance, user_dialogue_acts, other_features_from_dialogue_history2):
    utterance_history = other_features_from_dialogue_history2['utterance_history']
    acts_history = other_features_from_dialogue_history2['acts_history']
    prev_filled_slots = other_features_from_dialogue_history2['prev_filled_slots']
    return agent.predict_slots(user_utterance, utterance_history, acts_history, prev_filled_slots)
    #return  [('hotel-bookpeople', '2'), ('hotel-bookstay', '2'), ('hotel-bookday', 'sunday'), ('restaurant-phone', '?')]
    
def Information_to_be_retrieved_Prediction(user_dialogue_acts, extracted_information, other_features_from_dialogue_history3):
    print("TO BE RETRIEVED INPUT:")
    print(user_dialogue_acts)
    print(extracted_information)
    to_be_retrieved = set(['hotel-area', 'hotel-name', 'restaurant-name', 'restaurant-phone', 'restaurant-address', 'booking-ref'])
    return to_be_retrieved

def Agent_Move_Prediction(user_dialogue_acts, extracted_information, retrieved_information, other_features_from_dialogue_history4):
    agent_dialogue_acts = ["Booking-Request", "Hotel-Recommend", "Restaurant-Inform"]
    to_be_requested = set(['booking-bookday', 'booking-bookpeople', 'restaurant-pricerange'])
    return {"agent_dialogue_acts":agent_dialogue_acts,
            "to_be_requested":to_be_requested}

In [13]:
from collections import Counter


def print_output(*output):
    if do_print_dialogue_details:
        if len(output) == 1:
            print(output[0])
        elif len(output) == 2:
            print(output[0], output[1])
        elif len(output) == 3:
            print(output[0], output[1], output[2])
        elif len(output) == 4:
            print(output[0], output[1], output[2], output[3])
        else:
            print(output)

def count_matches(ground_truth_list, predicted_list):
    no_gt = len(ground_truth_list)
    no_predicted = len(predicted_list)
    no_correct = no_gt - sum((Counter(ground_truth_list) - Counter(predicted_list)).values())
    return no_gt, no_predicted, no_correct

def get_metrics(no_gt_global, no_predicted_global, no_correct_global):
    precision = 1.0*no_correct_global/no_predicted_global if no_predicted_global else 0.0
    recall = 1.0*no_correct_global/no_gt_global if no_gt_global else 0.0
    f1_score = 2.0*precision*recall/(precision+recall) if precision and recall else 0.0
    return precision, recall, f1_score

In [14]:
####
def process_intent_list(intent_list):
    intents = set()
    if len(intent_list) == 0:
        intents.add('other')
    for intent in intent_list:
        if intent.startswith('Restaurant'):
            intents.add(intent)
        elif intent.startswith('Hotel'):
            intents.add(intent)
        elif intent.startswith('general'):
            intents.add(intent)
        else:
            intents.add('other')
    # print(f'Original {intent_list}')
    # print(f'Modified {list(intents)}')
    return list(intents)
####

In [15]:
global do_print_dialogue_details
do_print_dialogue_details = True
print_ground_truth_structures = True
print_predicted = True

no_user_moves_gt_global = 0 # "gt" == "ground truth"
no_user_moves_predicted_global = 0
no_user_moves_correct_global = 0

no_user_info_gt_global = 0
no_user_info_predicted_global = 0
no_user_info_correct_global = 0

no_agent_info_to_retrieve_gt_global = 0
no_agent_info_to_retrieve_predicted_global = 0
no_agent_info_to_retrieve_correct_global = 0

no_agent_moves_gt_global = 0
no_agent_moves_predicted_global = 0
no_agent_moves_correct_global = 0

no_agent_info_to_request_gt_global = 0
no_agent_info_to_request_predicted_global = 0
no_agent_info_to_request_correct_global = 0

n_dialogues_to_evaluate = 10
n_evaluated = 0

nr = 0

for d,dial in enumerate(dataset['train']):
    if n_evaluated>=n_dialogues_to_evaluate:
        break
    # skip dialogues that are not in the hotel or restaurant domain
    if not any(set(dial['turns']['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(dial['turns']['utterance'])):
        continue
    print_output("Dialogue ID:", dial['dialogue_id'])
    compulsory_slots_hotel  = set(['hotel-bookpeople', 'hotel-bookstay', 'hotel-name', 'hotel-bookday']) # as an example, to be adjusted
    compulsory_slots_restaurant  = set(['restaurant-name']) # as an example, to be adjusted
    filled_slots = set()
    speaker_str = {0: 'User', 1: 'Agent'}
    turns = dial['turns']
    prev_filled_slots = {}
    for turn_id,utt in enumerate(turns['utterance']):
        speaker = speaker_str[turns['speaker'][turn_id]]
        if speaker == "User":
            #### Adding history
            if turn_id == 0:
                prev_user_utterance = ''
                prev_user_acts = []
                prev_bot_utterance = ''
                prev_bot_acts = []
            else:
                prev_user_utterance = turns['utterance'][turn_id - 2]
                prev_user_acts = process_intent_list(turns['dialogue_acts'][turn_id - 2]['dialog_act']['act_type'])
                prev_bot_utterance = turns['utterance'][turn_id - 1]
                prev_bot_acts = process_intent_list(turns['dialogue_acts'][turn_id - 1]['dialog_act']['act_type'])
            
            utterrance_history = [prev_user_utterance, prev_bot_utterance]
            acts_history = [prev_user_acts, prev_bot_acts]
            ####
            
            print_output("User's turn")
            print_output("User's utterance: " + utt)

            print_output("Extraction")

            indent = " "*4
            dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
            user_dialogue_acts_ground_truth = process_intent_list(dialogue_acts.copy())
            print_output(indent, "Dialogue acts:", dialogue_acts)

            if print_ground_truth_structures:
                print(user_dialogue_acts_ground_truth)

            if user_dialogue_acts_ground_truth:
                dialogue_acts_predicted = Dialogue_Act_Prediction(utt, other_features_from_dialogue_history = {
                    'utterance_history': utterrance_history,
                    'acts_history': acts_history,
                        })
                # evaluate user's dialogue acts
                no_gt, no_predicted, no_correct = count_matches(user_dialogue_acts_ground_truth, dialogue_acts_predicted)
                no_user_moves_gt_global += no_gt
                no_user_moves_predicted_global += no_predicted
                no_user_moves_correct_global += no_correct
                if print_predicted:
                    print(indent, "User's dialogue acts predicted:", dialogue_acts_predicted, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output(indent, "Extracted information:")
            print_output(indent, "Spans")
            extracted_information_not_mapped_ground_truth = []
            extracted_information_ground_truth = []
            extracted_information_per_dialogue_act_ground_truth = {}
            span_info = turns['dialogue_acts'][turn_id]['span_info']
            for span_i in range(len(span_info['span_start'])):
                act_type = span_info['act_type'][span_i]
                span_name = span_info['act_slot_name'][span_i]
                span_value = span_info['act_slot_value'][span_i]
                span_range = [span_info['span_start'][span_i], span_info['span_end'][span_i]]
                span_value_as_in_utterance = utt[span_info['span_start'][span_i]: span_info['span_end'][span_i]]
                print_output(indent*2, span_value + ("" if span_value_as_in_utterance==span_value else " ("+span_value_as_in_utterance+")"), span_range)
                if not act_type in extracted_information_per_dialogue_act_ground_truth:
                    extracted_information_per_dialogue_act_ground_truth[act_type] = []
                act_category = act_type.split("-")[0].lower()
                extracted_information_not_mapped_ground_truth.append(tuple([act_category+"-"+span_name, span_value_as_in_utterance]))
                if act_category in ["hotel", "restaurant", "general"]:
                    extracted_information_ground_truth.append(tuple([act_category+"-"+span_name, span_value]))
                extracted_information_per_dialogue_act_ground_truth[act_type].append(tuple([span_name, span_value]))

            print_output(indent, "Categorized information")
            slot_names_per_act = [slot['slot_name'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            slot_values_per_act = [slot['slot_value'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            for act_i in range(len(slot_names_per_act)):
                slot_names_values_per_act = [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i]))]
                print_output(indent*2, dialogue_acts[act_i], slot_names_values_per_act)
                if dialogue_acts[act_i].startswith("Hotel") or dialogue_acts[act_i].startswith("Restaurant"):
                    for slot_i in range(len(slot_names_per_act[act_i])):
                        if slot_names_per_act[act_i][slot_i] != "none" and slot_values_per_act[act_i][slot_i] == "?":
                            if not dialogue_acts[act_i] in extracted_information_per_dialogue_act_ground_truth:
                                extracted_information_per_dialogue_act_ground_truth[dialogue_acts[act_i]] = []
                            extracted_information_not_mapped_ground_truth.append(tuple([dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            extracted_information_ground_truth.append(tuple([dialogue_acts[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))
                            print(extracted_information_ground_truth[-1])
                            extracted_information_per_dialogue_act_ground_truth[dialogue_acts[act_i]].append(tuple([slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))

            if print_ground_truth_structures:
                print(extracted_information_not_mapped_ground_truth)
                print(extracted_information_ground_truth)
                print(extracted_information_per_dialogue_act_ground_truth)

            if user_dialogue_acts_ground_truth:
                extracted_information = Extract_and_Categorize_Spans(utt, user_dialogue_acts_ground_truth, other_features_from_dialogue_history2={
                    'utterance_history': utterrance_history,
                    'acts_history': acts_history,
                    'prev_filled_slots': prev_filled_slots,
                        })
                # evaluate information extraction only if there are Hotel or Restaurant or general dialogue acts
                no_gt, no_predicted, no_correct = count_matches(extracted_information_ground_truth, extracted_information)
                if any(da.startswith("general") or da.startswith("Hotel") or da.startswith("Restaurant") for da in user_dialogue_acts_ground_truth):
                    no_user_info_gt_global += no_gt
                    no_user_info_predicted_global += no_predicted
                    no_user_info_correct_global += no_correct
                    if print_predicted:
                        print(indent, "Extracted and categorized information (predicted):", extracted_information, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output("Reasoning (dialogue state tracking)")
            services = turns['frames'][turn_id]['service']
            print_output(indent, "Services:", services)
            current_booking_service = [service for service in services if service in ["hotel", "restaurant"]]

            not_empty_intents = [intent for intent in turns['frames'][turn_id]['state'] if intent['requested_slots'] or intent['slots_values']['slots_values_name']]
            if not_empty_intents:
                print_output(indent, "Intents")
                for intent in not_empty_intents:
                    print_output(indent*2, "Active intent:", intent['active_intent'])
                    requested_slots = intent['requested_slots']
                    if requested_slots:
                        print_output(indent*2, "Requested slots:", requested_slots)
                    if intent['slots_values']['slots_values_name']:
                        slot_names = intent['slots_values']['slots_values_name']
                        slot_values = intent['slots_values']['slots_values_list']
                        filled_slots.update(slot_names)
                        print_output(indent*2, "Filled slots:")
                        for slot_i in range(len(slot_names)):
                            print_output(indent*3, slot_names[slot_i]+": ", slot_values[slot_i])
                    print_output(indent*2, "--------------")

            print_output(indent, "Missing slots (Hotel):", compulsory_slots_hotel - filled_slots)
            print_output(indent, "Missing slots (Restaurant):", compulsory_slots_restaurant - filled_slots)
        elif speaker == "Agent":
            nr += 1
            print("AGENT NR", nr)
            indent = " "*4
            print_output("Agent's turn")
            dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']

            do_evaluate_agent_turn = True
            if not any(da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") for da in dialogue_acts):
                do_evaluate_agent_turn = False
                print_output("This agent's turn won't be evaluated as it is out of domain.")

            print_output("Retrieval")

            slot_names_per_act = [slot['slot_name'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]
            slot_values_per_act = [slot['slot_value'] for slot in turns['dialogue_acts'][turn_id]['dialog_act']['act_slots']]

            to_be_retrieved_ground_truth = []
            print_output(indent, "Information to be retrieved:")
            for act_i in range(len(slot_names_per_act)):
                domain = dialogue_acts[act_i].split("-")[0].lower()
                if domain == "booking" and len(current_booking_service)==1:
                    domain = current_booking_service[0]
                slot_names = [domain+"-"+slot_names_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i]))
                              if slot_values_per_act[act_i][slot_i]!="?" and slot_names_per_act[act_i][slot_i]!="none"]
                
                if slot_names:
                    to_be_retrieved_slot_names = ["%s-availability" % (domain)] + slot_names
                    while domain+"-choice" in to_be_retrieved_slot_names:
                        del to_be_retrieved_slot_names[to_be_retrieved_slot_names.index(domain+"-choice")]
                    to_be_retrieved_ground_truth.extend(to_be_retrieved_slot_names)
            to_be_retrieved_ground_truth = sorted(list(set(to_be_retrieved_ground_truth)))
            print_output(indent*2, "To be retrieved:", to_be_retrieved_ground_truth, "<--- That's the first thing we predict in agent's move.")

            to_be_provided_overall = []
            print_output(indent, "Retrieved information:")
            for act_i in range(len(slot_names_per_act)):
                domain = dialogue_acts[act_i].split("-")[0].lower()
                if domain == "booking" and len(current_booking_service)==1:
                    domain = current_booking_service[0]
                if domain in ["hotel", "restaurant", "booking", "general"]:
                    slot_names_vlues = [domain+"-"+slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i]))
                                        if slot_values_per_act[act_i][slot_i]!="?" and slot_names_per_act[act_i][slot_i]!="none"]
                    if slot_names_vlues and any((slot_name_value.split(":")[0]!=domain+"-none" for slot_name_value in slot_names_vlues)) and not "-No" in dialogue_acts[act_i]:
                        to_be_provided = ["%s-availability:yes" % (domain)] + slot_names_vlues
                        to_be_provided_overall.extend(to_be_provided)
                    elif "-No" in dialogue_acts[act_i]:
                        to_be_provided = ["%s-availability:no" % (domain)] + slot_names_vlues
                        to_be_provided_overall.extend(to_be_provided)
            to_be_provided_overall = sorted(list(set(to_be_provided_overall)))
            remove_avail_no_list = [elem for elem in to_be_provided_overall if elem.endswith("availability:no")]
            for remove_avail in remove_avail_no_list:
                remove_avail_yes = remove_avail[:-2]+"yes"
                while remove_avail_yes in to_be_provided_overall:
                    del to_be_provided_overall[to_be_provided_overall.index(remove_avail_yes)]
            print_output(indent*2, "Retrieved info to be provided:", to_be_provided_overall)

            print_output("Planning")
            agent_dialogue_acts_ground_truth = []
            for act_i in range(len(slot_names_per_act)):
                domain = dialogue_acts[act_i].split("-")[0].lower()
                if domain in ["hotel", "restaurant", "booking", "general"]:
                    agent_dialogue_acts_ground_truth.append(dialogue_acts[act_i])
            print_output(indent, "Agent's move (dialogue acts):", agent_dialogue_acts_ground_truth, "<--- That's the second thing we predict in agent's move.")

            print_output(indent, "Information to be requested:")
            to_be_requested_ground_truth = []
            for act_i in range(len(slot_names_per_act)):
                domain = dialogue_acts[act_i].split("-")[0].lower()
                if domain == "booking" and len(current_booking_service)==1:
                    domain = current_booking_service[0]
                if domain in ["hotel", "restaurant", "booking", "general"]:
                    to_be_requested = [domain+"-"+slot_names_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]=="?"]
                    to_be_requested_ground_truth.extend(to_be_requested)
            to_be_requested_ground_truth = sorted(list(set(to_be_requested_ground_truth)))
            print_output(indent*2, "To be requested:", to_be_requested_ground_truth, "<--- That's the third thing we predict in agent's move.")

            print_output(indent, "Planned move per dialogue act (we won't evaluate this):")
            for act_i in range(len(slot_names_per_act)):
                print_output(indent*2, dialogue_acts[act_i])
                print_output(indent*2, "To be provided:", [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]!="?"])
                print_output(indent*2, "To be requested:", [slot_names_per_act[act_i][slot_i]+":"+slot_values_per_act[act_i][slot_i] for slot_i in range(len(slot_names_per_act[act_i])) if slot_values_per_act[act_i][slot_i]=="?"])
                print_output(indent*2, "--------------")

            print("TO BE RETRIEVED TRUTH:", to_be_retrieved_ground_truth)
            agent_move_ground_truth = {
                "to_be_retrieved": to_be_retrieved_ground_truth,  # set, only unique names of the slots
                "dialogue_acts": agent_dialogue_acts_ground_truth,
                "to_be_requested": to_be_requested_ground_truth,  # set, only unique names of the slots
                "retrieved_information_per_dialogue_act": {} # non-none names of the slots and values grouped per dialogue act
                }

            for act_i in range(len(slot_names_per_act)):
                da = dialogue_acts[act_i]
                if da.startswith("Hotel") or da.startswith("Restaurant") or da.startswith("Booking") or da.startswith("general"):
                    for slot_i in range(len(slot_names_per_act[act_i])):
                        if slot_names_per_act[act_i][slot_i] != "none":
                            if not dialogue_acts[act_i] in agent_move_ground_truth["retrieved_information_per_dialogue_act"]:
                                agent_move_ground_truth["retrieved_information_per_dialogue_act"][dialogue_acts[act_i]] = []
                            agent_move_ground_truth["retrieved_information_per_dialogue_act"][dialogue_acts[act_i]].append(tuple([slot_names_per_act[act_i][slot_i], slot_values_per_act[act_i][slot_i]]))

            if do_evaluate_agent_turn and user_dialogue_acts_ground_truth:
                agent_to_be_retrieved_predicted = Information_to_be_retrieved_Prediction(user_dialogue_acts_ground_truth,
                                                                                         extracted_information_per_dialogue_act_ground_truth,
                                                                                         other_features_from_dialogue_history3={})
                agent_move_predicted = Agent_Move_Prediction(user_dialogue_acts_ground_truth,
                                                             extracted_information_per_dialogue_act_ground_truth,
                                                             to_be_provided_overall,
                                                             other_features_from_dialogue_history4={})
                if print_ground_truth_structures:
                    print(agent_move_ground_truth)

                if print_predicted:
                    print("Planning predicted")

                # evaluate "to_be_retrieved" only if there are Hotel or Restaurant or Booking or general dialogue acts
                no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["to_be_retrieved"], agent_to_be_retrieved_predicted)
                no_agent_info_to_retrieve_gt_global += no_gt
                no_agent_info_to_retrieve_predicted_global += no_predicted
                no_agent_info_to_retrieve_correct_global += no_correct
                if print_predicted:
                    print(indent, "Info to be retrieved predicted:", agent_to_be_retrieved_predicted, "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

                # evaluate agent's dialogue acts
                no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["dialogue_acts"], agent_move_predicted["agent_dialogue_acts"])
                no_agent_moves_gt_global += no_gt
                no_agent_moves_predicted_global += no_predicted
                no_agent_moves_correct_global += no_correct
                if print_predicted:
                    print(indent, "Agent's dialogue acts predicted:", agent_move_predicted["agent_dialogue_acts"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

                # evaluate "retrieved_information" -> "to_be_requested" only if there are Hotel or Restaurant or Booking or general dialogue acts
                no_gt, no_predicted, no_correct = count_matches(agent_move_ground_truth["to_be_requested"], agent_move_predicted["to_be_requested"])
                no_agent_info_to_request_gt_global += no_gt
                no_agent_info_to_request_predicted_global += no_predicted
                no_agent_info_to_request_correct_global += no_correct
                if print_predicted:
                    print(indent, "Info to be requested predicted:", agent_move_predicted["to_be_requested"], "correct = %d/%d, true covered = %d/%d" % (no_correct, no_predicted, no_correct, no_gt))

            print_output("Agent's utterance: "+utt)
        print_output("-------------------------------------------------------------------")
        print_output("-------------------------------------------------------------------")

    n_evaluated+=1

print("Dialogue acts in the user's move prediction")
precision, recall, f1_score = get_metrics(no_user_moves_gt_global, no_user_moves_predicted_global, no_user_moves_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Extracted information from user's utterance")
precision, recall, f1_score = get_metrics(no_user_info_gt_global, no_user_info_predicted_global, no_user_info_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Info to be retrieved by the agent")
precision, recall, f1_score = get_metrics(no_agent_info_to_retrieve_gt_global, no_agent_info_to_retrieve_predicted_global, no_agent_info_to_retrieve_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Dialogue acts in the agent's move prediction")
precision, recall, f1_score = get_metrics(no_agent_moves_gt_global, no_agent_moves_predicted_global, no_agent_moves_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))
print("Info to be requested by the agent")
precision, recall, f1_score = get_metrics(no_agent_info_to_request_gt_global, no_agent_info_to_request_predicted_global, no_agent_info_to_request_correct_global)
print("Precision: %lf, Recall: %lf, F1-score: %lf" % (precision, recall, f1_score))

Dialogue ID: PMUL4398.json
User's turn
User's utterance: i need a place to dine in the center thats expensive
Extraction
     Dialogue acts: ['Restaurant-Inform']
['Restaurant-Inform']
     User's dialogue acts predicted: ('Restaurant-Inform',) correct = 1/1, true covered = 1/1
     Extracted information:
     Spans
         centre (center) [30, 36]
         expensive [43, 52]
     Categorized information
         Restaurant-Inform ['area:centre', 'pricerange:expensive']
[('restaurant-area', 'center'), ('restaurant-pricerange', 'expensive')]
[('restaurant-area', 'centre'), ('restaurant-pricerange', 'expensive')]
{'Restaurant-Inform': [('area', 'centre'), ('pricerange', 'expensive')]}
     Extracted and categorized information (predicted): [('restaurant-area', 'centre'), ('restaurant-pricerange', 'expensive')] correct = 2/2, true covered = 2/2
Reasoning (dialogue state tracking)
     Services: ['restaurant', 'hotel']
     Intents
         Active intent: find_restaurant
         Filled s